# Exemple de scraping du site Lumiplan William Gainnier DIA1 :

Etape 1 : Scraper tout les liens contenant les informations relatives à chaque station:

In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.lumiplan.com/produit/les-stations-temps-reel/' 

# envoie d'une requete get a à lumiplan
response = requests.get(url,verify=False)

# Check si la requete à eu une réponse:
if response.status_code == 200:

    soup = BeautifulSoup(response.content, 'html.parser')
    
    # On récupere tout les liens hrefs de la page (cette page contient tout les liens des stations)
    links = [link.get('href') for link in soup.find_all('a') if link.get('href')and (link.get('href').startswith('https://bulletin.lumiplan') or link.get('href').startswith('https://bulletinv3.lumiplan')) ]
    
else:
    print('Failed to retrieve the web page')


C:\Users\willi\anaconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
C:\Users\willi\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.lumiplan.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Etape 2 : Scraper le contenu de chaque lien obtenu précedemment

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
from datetime import date
# On récupere la date d'aujourd'hui afin d'inclure dans le dataset final la date du scraping.
today_date = date.today()
warnings.filterwarnings('ignore')
count= 1
df =pd.DataFrame()  


for url in links:
    print(count)
    my_dict = {}
    #Le site lumiplan a 2 types de sites, les sites avec l'url bulletinv3 et les sites bulletin nous ferons 1 scraper different pour chaque car ils sont construit différemment
    if "bulletinv3" in url:
            remontee = []
            pistes_activitee =[]
            meteolist = []
            station_name = url.split('station=')[1].split('&')[0]
            
            response = requests.get(url, verify=False)
            
            
            # Check si la requete à eu une réponse:
            if response.status_code == 200:
                #On parcourt le premier bloc qui contient les informations relatives aux différentes zone de la station
                soup = BeautifulSoup(response.content, 'html.parser')
                div_elements = soup.find_all('div', class_='bloc_pistes_remontees')
                #On recupere les différents bloc contenant les informations relatives aux pistes et remontées de chaque station de chaque zone
                for div in div_elements:
                    meteo_cards = div.find_all('div',class_="meteo_card")
                    for meteo in meteo_cards:
                        meteolist.append(meteo)
                    
                    #on parcourt les blocs de chaque zone
                    for card in meteolist:
                        h2_elements = card.find_all('h2')
                        blocremontee = card.find_all('div', class_="card-content remontee_pistes")
                        #On récupere le nom de chaque zone
                        for h2 in h2_elements:
                            h2text = h2.text.strip()
                            poi_titles = {}
                            for item in blocremontee:
                                #On récupere l'information POI_title qui permet de savoir si c'est une remontée mécanique ou une piste/activitée
                                poi_title_elements = item.find_all('div', class_='POI_title')
                                liaisons_elements = item.find_all('div', class_='liaisons')
                                
                                #On parcourt chaque bloc contenant les informations relatives aux remontées mécaniques et aux pistes/activitées
                                for poi_title, liaisons in zip(poi_title_elements, liaisons_elements):
                                    poi_title_text = poi_title.text.strip()
                                    
                                    #On y recupere les noms des remontées et pistes
                                    nom_message = liaisons.find_all('div', class_='nom_message')
                                    #les images correspondant qui indique si la remontée/piste est fermée ou non
                                    picto_imgs = liaisons.find_all('div', class_='picto')
                                    #On récupere l'image indiquant le type d'activités si c'est une piste de ski de randonnée de luges etc...
                                    type_pistes = liaisons.find_all('div', class_= 'POI_info')
                                    
                                    listetup = []
                                    
                                    for nom,picto_img,type_piste in zip(nom_message,picto_imgs,type_pistes):
                                        #On recupere du lien de l'image l'info de si la piste est ouverte ou non ou non indiquée
                                        picto_img = picto_img.find('img', alt='statut ouverture')
                                        picto_src = picto_img['src'] if picto_img else None
                                        
                                        picto_src_value = (
                                            0 if 'closed' in picto_src else
                                            1 if 'opened' in picto_src else
                                            2 if 'period' in picto_src else
                                            3
                                        )
                                        type_piste = type_piste.find('img')['src'] if type_piste else None
                                        
                                        #idem ici mais pour les couleurs des pistes
                                        couleur = (
                                            0 if 'GREEN' in type_piste else
                                            1 if 'BLUE' in type_piste else
                                            2 if 'RED' in type_piste else
                                            3 if 'BLACK' in type_piste else
                                            4 if 'YELLOW' in type_piste else
                                            None
                                        )
                                        #idem ici mais pour connaitre le type d'activitée
                                        ski = (
                                            "SKI" if 'DOWNHILL' in type_piste else
                                            "Cross_country" if 'CROSS_COUNTRY' in type_piste else
                                            "SNOWSHOE" if 'SNOWSHOE' in type_piste else
                                            "SNOWPARK" if 'SNOWPARK' in type_piste else
                                            "BOARDERCROSS" if 'BOARDERCROSS' in type_piste else
                                            "SKI_TOURING" if 'SKI_TOURING' in type_piste else
                                            "FUN_ZONE" if 'FUN_ZONE' in type_piste else
                                            "SLEDDING_HILL" if 'SLEDDING_HILL' in type_piste else
                                            "Other"
                                        )
                                        #enfin on ajoute dans un tuple le nom de la piste/remontée si elle est ouverte sa couleur et son type d'activitée
                                        
                                        if 'Pistes' in poi_title_text:
                                            tup = (nom.text.strip(),picto_src_value,couleur,ski)
                                            
                                        if 'Remontées' in poi_title_text:
                                            tup = (nom.text.strip(),picto_src_value,"remontée")
                                            
                                        #on ajoute dans une liste a part pour avoir acces directement dans le dataframe à toutes les pistes/activitées de TOUTE la station
                                        if 'Pistes' in poi_title_text:
                                            
                                            pistes_activitee.append(tup)
                                        
                                        #idem ici mais pour les remontées
                                        if 'Remontées' in poi_title_text:
                                            remontee.append(tup)
                                        if 'Liaiso' in poi_title_text:
                                            remontee.append(tup)
                                        
                                        #ici a contrario on mets les tup dans une liste qu'on associe à sa zone correspondante afin d'avoir l'infromation détaillée.
                                        listetup.append(tup)
                                    poi_titles[poi_title_text] = listetup

                            my_dict[h2text] = poi_titles


                df =df.append({'station_name': station_name,'url': url,'Pistes':pistes_activitee,'Remontées':remontee, 'detail': my_dict,"scrap_date":
        today_date}, ignore_index=True)
                count +=1
            else:
                print('Failed to retrieve the web page')
                df= df.append({'station_name': station_name,'url': url,'Pistes':'fail','Remontées':'fail', 'detail': 'fail', "scrap_date":today_date}, ignore_index=True)
                count +=1
                
    #Le site lumiplan a 2 types de sites, les sites avec l'url bulletinv3 et les sites bulletin nous ferons 1 scraper different pour chaque car ils sont construit différemment
    #le scraping du deuxieme type de site suit une logique tres similaire a celui juste au dessus on ne reviendra pas sur l'explication de cette partie qui serait redondante
    else:
        remontee = []
        pistes_activitee =[]
        station_name = url.split('station=')[1].split('&')[0]
        
        response = requests.get(url, verify=False)
       
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            my_dict = {}
            div_elements = soup.find_all('div', class_='pistes_remontees')
            
            for div in div_elements:
                h2_elements = div.find_all('div', class_='sector center col-12')
                blocremontee = div.find_all('div', class_="card-body prl_info")
                
                for h2 in h2_elements:
                    h2text = h2.text.strip()
                    poi_titles = {}
                    for item in blocremontee:
                        poi_title_elements = item.find_all('div', class_='text col-12')
                        liaisons_elements = item.find_all('div', class_='row prl_affichage')

                        for poi_title, liaisons in zip(poi_title_elements, liaisons_elements):
                            poi_title_text = poi_title.text.strip()

                
                            nom_message = liaisons.find_all('div', class_='col-lg-3 col-3 prl_name')
                            picto_imgs = liaisons.find_all('div', class_='col-lg-1 col-1 status')
                            
                            type_pistes = liaisons.find_all('div', class_= 'col-lg-1 col-1 prl_center')
                            
                            
                            listetup = []
                            for nom,picto_img,type_piste in zip(nom_message,picto_imgs,type_pistes):
                                picto_src = picto_img.find('img')['src'] if picto_img else None
                                picto_src_value = (
                                    0 if 'F' in picto_src else
                                    1 if 'O' in picto_src else
                                    2 if 'P' in picto_src else
                                    3 if 'H' in picto_src else
                                    None
                                )
                                type_piste = type_piste.find('img')['src'] if type_piste else None 
                                couleur = (
                                    0 if 'V' in type_piste else
                                    1 if 'B' in type_piste else
                                    2 if 'R' in type_piste else
                                    3 if 'N' in type_piste else
                                    None
                                )
                                ski=(
                                    "SKI" if 'A-V' in type_piste else
                                    "SKI" if 'A-B' in type_piste else
                                    "SKI" if 'A-R' in type_piste else
                                    "SKI" if 'A-N' in type_piste else
                                    "Fun_Zone" if 'ZL' in type_piste else
                                    "SLEDDING_HILL" if 'L-' in type_piste else
                                    "PE" if 'PE-' in type_piste else
                                    "SNOWPARK" if 'S-' in type_piste else
                                    "Other" if 'S-' in type_piste else
                                    
                                    None
                                )
                                
                                if 'Pistes' in poi_title_text:
                                    tup = (nom.text.strip(),picto_src_value,couleur,ski)
                                            
                                if 'Remontées' in poi_title_text:
                                    tup = (nom.text.strip(),picto_src_value,"remontée")
                                
                               
                                if 'Pistes' in poi_title_text:
                                    
                                    pistes_activitee.append(tup)
                                if 'Remontées' in poi_title_text:
                                    remontee.append(tup)

                                listetup.append(tup)
                            poi_titles[poi_title_text] = listetup

                    my_dict[h2text] = poi_titles


                df =df.append({'station_name': station_name,'url': url,'Pistes':pistes_activitee,'Remontées':remontee, 'detail': my_dict,"scrap_date":
        today_date}, ignore_index=True)
                count +=1
        else:
            print('Failed to retrieve the web page')
            df= df.append({'station_name': station_name,'url': url,'Pistes':'fail','Remontées':'fail', 'detail': 'fail', "scrap_date":today_date}, ignore_index=True)
            count +=1
            


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


La colonne piste contient [(nompiste,ouvert/fermé,couleur,type_piste),...]
La colonne remontées idem [(nomremontées/fermé,couleur,type_piste),...]
la colonne détail contient les informations relatives a chaque sous domaine d'une station donc les pistes et remontées de chaque sous domaine

In [3]:
df.to_csv("Lumiplan_scrap_18.csv")